#### 국내기후자료평년값자료(30년)
http://www.weather.go.kr/weather/climate/average_30years.jsp

평년 기후 자료를 시군 별로 저장한 일별 데이터 

HTML 구조도 어렵지 않아서 간단히 구현하였다

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import csv

In [3]:
url = "http://www.weather.go.kr/weather/climate/average_30years.jsp?"

params = {
    "yy_st":'2018',
    "mm":'1', #월
    "dd":'1' #일
}

In [4]:
resp = requests.get(url, params) #requests 모듈을 이용해 url과 params 값으로 get 방식 request 한다

In [5]:
resp.encoding  = 'euc-kr' #인코딩을 명시한다

In [6]:
resp.text[:1000] #str

'\n\n\n\n\n\n\n\t<!DOCTYPE html>\n\t<!-- <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"> -->\n\t<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" lang="ko">\n\t<head>\n\t\t<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n\t\t<title>평년값자료(30년) &gt; 국내기후자료 &gt; 기후자료 &gt; 날씨 &gt; 기상청 </title>\n        \t<link rel="shortcut icon" href="http://www.kma.go.kr/favicon2.ico" />\n\t\t<meta http-equiv="Content-Type" content="text/html; charset=euc-kr" />\n\t\t<link rel="stylesheet" type="text/css" href="/share/css/base.css?ver=2017121701" />\n\t\t<link rel="stylesheet" type="text/css" href="/share/css/common.css?ver=2017121701" />\n\t\t<link rel="stylesheet" type="text/css" href="/share/css/weather-layout.css?ver=20171217" />\n\t\t<link rel="stylesheet" type="text/css" href="/share/css/component.css?ver=20180326" />\n\t\t\n\t\t\n\t\t\n\t\t\t<link rel="stylesheet" type="text/css" href="/share/css/weather-obs.c

In [8]:
soup = BeautifulSoup(resp.text,"lxml") #가져온 스트링을 BeautifulSoup사용해 파싱한다

In [10]:
weathers = soup.select("table.table_develop > tbody > tr") #css selector를 사용하여 태그와 텍스트를 리스트로 가져올 수 있다
weathers[0]

<tr>
<td> 090 속초</td>
<td>0.5</td>
<td>4.6</td>
<td>-3.3</td>
<td>0.9</td>
<td>2.0</td>
<td>3.3</td>
<td>49.0</td>
<td>3.3</td>
<td>5.8</td>
<td>3.3</td>
<td>1020.3</td>
</tr>

In [65]:
weathers = soup.find('table',{'class':'table_develop'}).find('tbody').find_all('td') #find와 find_all 이용해서도 역시 쉽게 찾는다
weathers[0].text.strip()

'090 속초'

In [11]:
weatherLst = [weather.get_text().strip().split('\n') for weather in weathers] # strip으로 공백을 제거했고 split을 이용하여 변수별로 잘라 리스트를 만들었다
weatherLst[0] 

['090 속초',
 '0.5',
 '4.6',
 '-3.3',
 '0.9',
 '2.0',
 '3.3',
 '49.0',
 '3.3',
 '5.8',
 '3.3',
 '1020.3']

In [66]:
#이상을 함수화하면 다음과 같다
def weather_crawler(url, month, day): 
    params = {
        "yy_st":"2018",
        "mm":str(month),
        "dd":str(day)
    }
    
    resp = requests.get(url, params)
    resp.encoding  = 'euc-kr'
    
    weathers = BeautifulSoup(resp.text,"lxml").select("table.table_develop > tbody > tr")
    weatherLst = [weather.get_text().strip().split('\n') for weather in weathers]
    
    return weatherLst

In [83]:
url = "http://www.weather.go.kr/weather/climate/average_30years.jsp?"

params = {
    "yy_st":'2018',
    "mm":'1',
    "dd":'1'
}

In [85]:
#csv 파일로 써준다
#결측치와 인코딩 문제 해결을 위한 간단한 코딩이 필요하다

fp = open('weather_data2.csv','w', encoding='euc-kr',newline='')
wr = csv.writer(fp)

for m in range(1,13):
    for d in range(1,32):
        if (m in [4,6,9,11]) and (d==31): #존재하지 않는 날짜를 param으로 넣은 경우에도 링크 존재하므로, 조건문 사용해서 걸러준다
            continue
        elif (m==2) and (d>29):
            continue
            
        try:
            weatherLst = weather_crawler(url,m,d)
            for weather in weatherLst:
                while len(weather)<12: #결측치있는 경우 nan 넣어줌
                    weather.append(np.nan)
                                   
                weather.append("%d월 %d일" % (m,d)) #날짜 넣어줌
                
                while '\xa0' in weather: # 인코딩 관련 에러 해결위해 
                    weather[weather.index('\xa0')] = np.nan
                    
                wr.writerow(weather) # 한줄씩 write
                
        except Exception as e:
            print(e)
            print(m, "월", d, "일", 'exception exists')
            continue
            
fp.close()